# Importing libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import time
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

# 讀取檔案

In [2]:
training_data = pd.read_csv('adult.data', sep=", ", header=None, engine='python')
test_data = pd.read_csv('adult.test', sep=", ", header=None, engine='python', skiprows=1)
training_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
training_data.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
test_data.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
feature_cols = ['age','workclass','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','native-country','income']
training_data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## Data Cleaning

In [4]:
## education跟education-num數量一樣所以取education-num即可，故刪除education
training_data = training_data.drop(columns=['education'])
test_data = test_data.drop(columns=['education'])

## fnlwgt 是ID，故刪除
training_data = training_data.drop(columns=['fnlwgt'])
test_data = test_data.drop(columns=['fnlwgt'])

## get_dummies

In [5]:
convert = {"income":{"<=50K":0, ">50K":1}, "sex":{"Male":1,"Female":0}}
training_data = training_data.replace(convert)
convert = {"income":{"<=50K.":0, ">50K.":1}, "sex":{"Male":1,"Female":0}}
test_data = test_data.replace(convert)
training_data

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,0
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,0
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,0
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,0
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,12,Married-civ-spouse,Tech-support,Wife,White,0,0,0,38,United-States,0
32557,40,Private,9,Married-civ-spouse,Machine-op-inspct,Husband,White,1,0,0,40,United-States,1
32558,58,Private,9,Widowed,Adm-clerical,Unmarried,White,0,0,0,40,United-States,0
32559,22,Private,9,Never-married,Adm-clerical,Own-child,White,1,0,0,20,United-States,0


In [6]:
from sklearn.preprocessing import MinMaxScaler
# numerical_columns
num_columns = ['age','education-num','capital-gain','capital-loss']
scaler = MinMaxScaler()

training_data_scaled = pd.DataFrame(scaler.fit_transform(training_data[num_columns]), columns=num_columns)
training_data[num_columns] = training_data_scaled

test_data_scaled = pd.DataFrame(scaler.transform(test_data[num_columns]), columns=num_columns)
test_data[num_columns] = test_data_scaled

training_data.describe()

,age,education-num,sex,capital-gain,capital-loss,hours-per-week,income
count,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,0.295639,0.605379,0.669205,0.010777,0.020042,40.437456,0.240810
std,0.186855,0.171515,0.470506,0.073854,0.092507,12.347429,0.427581
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.150685,0.533333,0.000000,0.000000,0.000000,40.000000,0.000000
50%,0.273973,0.600000,1.000000,0.000000,0.000000,40.000000,0.000000
75%,0.424658,0.733333,1.000000,0.000000,0.000000,45.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,99.000000,1.000000


In [7]:
training_data.replace('?', 'other', inplace=True)
test_data.replace('?', 'other', inplace=True)

In [8]:
training_data = pd.get_dummies(training_data)
test_data = pd.get_dummies(test_data)
feature_cols = training_data.columns
for column in training_data.columns:
    if column not in test_data.columns:
        test_data[column] = 0
        break

In [9]:
X = training_data.drop('hours-per-week',axis=1)
y = training_data['hours-per-week']
Xt = test_data.drop('hours-per-week',axis=1)
yt = test_data['hours-per-week']
X.columns = Xt.columns

## MAPE、RMSE、R$^2$Score

In [10]:
from sklearn.metrics import mean_absolute_percentage_error
model = RandomForestRegressor()
start_time = time.time()
model.fit(X, y)
print(f'Spend time: {time.time()-start_time}')
y_pred =  model.predict(Xt)
print(f'MAPE: {mean_absolute_percentage_error(yt,y_pred)}')
rmse = np.sqrt(metrics.mean_squared_error(yt,y_pred))
print(f'RMSE: {rmse}')
print(f'r2_score : {r2_score(yt,y_pred)}')

Spend time: 48.54167675971985
MAPE: 0.31730762578232236
RMSE: 11.483026924812924
r2_score : 0.15324699385310758
